# Radiative transfer

To test whether the radiative transfer calculation is working correctly, I will examine a model with erroneous intensity calculations (with some sightlines being $~10^{40}K$). Below we can see how the whole sky map can be narrowed to just a few erroneous sightlines.

There is also an option when evaluating the refined sightlines to save the intermediate arrays in numpy files (`I.npy`, `e.npy`, `de.npy`, `k.npy`, `dk.npy`, `a.npy`, and `b.npy`, corresponding to $I_\nu$, $\epsilon_\nu$, $\Delta \epsilon_\nu$, $\kappa_\nu$, $\Delta \kappa_\nu$, $a_\nu$, and $b_\nu$, respectively) to load and work with the data.

In [6]:
from pprint import pprint

import numpy as np
from astropy.io import fits

In [3]:
chmap = fits.open(r"c:\users\cyani\projects\pdr\KT3_history\MilkyWay\r400_cm1-1_d1_uv10\channel_intensity.fits")

In [5]:
chmap[1].shape

(701, 5, 180, 18)

In [7]:
pprint(chmap[1].header)

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                  -64 / array data type                                
NAXIS   =                    4 / number of array dimensions                     
NAXIS1  =                   18                                                  
NAXIS2  =                  180                                                  
NAXIS3  =                    5                                                  
NAXIS4  =                  701                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
TYPE    = 'Species transitions'                                                 
BUNIT   = 'K       '                                                            
CTYPE1  = 'Wavelength'                                                          
CUNIT1  = 'm       '        

In [4]:
np.where(chmap[1].data[:,:,:,0]>10**30)

(array([341, 341, 341, 341, 359, 359, 359, 359], dtype=int64),
 array([2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([170, 171, 172, 173,   6,   7,   8,   9], dtype=int64))

In [9]:
lon = np.linspace(chmap[1].header['CRVAL2'] - chmap[1].header['CDELT2']*chmap[1].header['CRPIX2'],
                  chmap[1].header['CRVAL2'] + chmap[1].header['CDELT2']*chmap[1].header['CRPIX2'],
                  num=chmap[1].header['NAXIS2'])

In [11]:
lon[170],lon[173]

(2.8414642208742613, 2.9473572974285815)

### Theory

In order to test the correct calculation of radiative transfer, the theory must first be covered. The radiative transfer (RT) equation,

$$
dI_\nu = - I_\nu \kappa_\nu ds + \epsilon_\nu ds
$$

must be integrated to calculate the observed intensity. Simple applications of RT assume constant $\epsilon_\nu$ and $\kappa_\nu$ between voxels to obtain the observed intensity. The benefit of `kosmatau3d` is the more accurate calculation of the RT equation. Here the emissivity and absorption terms, $\epsilon_\nu$ and $\kappa_\nu$, are still used, but the first-order linear terms, $\Delta\epsilon_\nu = \frac{\epsilon_\nu}{\Delta s}$ and $\Delta\kappa_\nu = \frac{\kappa_\nu}{\Delta s}$, also appear. Here $\Delta s$ is the width of each voxel. This allows us to define specific cases for the way we integrate the RT equation:

 - *Basic*: $\Delta\kappa = 0$ and $| \kappa \Delta s | < 10^{-10}$
 - *Simple*: $\kappa > 10^3 \Delta \kappa \Delta s$
 - *Complex*: otherwise
   - *method 1*: $\Delta \kappa > 0$
   - *method 2*: $\Delta \kappa < 0$
   
The complex methods of calculating the RT equation are the most accurate, and should be used by default. The other methods are simplifications that can be made. The *Basic* case occurs when there is (mostly) no absorption. Since $\kappa_\nu = 0$ and $\Delta\kappa_\nu = 0$, the RT equation becomes, 

$$
\int_0^{I_\nu} dI_\nu = \int_0^{\Delta s} \left( \epsilon_\nu + \Delta\epsilon_\nu s' \right) ds', \\
I_\nu = \epsilon_\nu \Delta s + \frac{1}{2} \Delta\epsilon_\nu \left( \Delta s \right)^2 + I_{\nu,0}.
$$

The *Simple* case occurs when there is constant absorption, so $\Delta\kappa_\nu = 0$ and $\kappa_\nu \neq 0$. This has a more-complex equation:

$$
\int_0^{I_\nu} dI_\nu = e^{-\kappa_\nu \Delta s} \left( \int_0^{\Delta s} (\epsilon_\nu + \Delta \epsilon_\nu s') e^{\kappa_\nu s'} ds' + I_0 \right), \\
I_\nu = \left( \frac{\epsilon_\nu \kappa_\nu + \Delta\epsilon_\nu (\kappa_\nu \Delta s - 1)}{\kappa_\nu^2} \right) - e^{-\kappa_\nu \Delta s} \left( \frac{\epsilon_\nu \kappa_\nu - \Delta\epsilon_\nu}{\kappa_\nu^2} \right) + e^{-\kappa_\nu \Delta s} I_{\nu,0}.
$$

Finally, the *Complex* integration is the full unsimplified integration of the RT equation. The only distinction made is if $\Delta\kappa \lessgtr 0$.

$$
I_\nu = \frac{\Delta\epsilon_\nu}{\Delta\kappa_\nu} \left( 1 - e^{-\kappa_\nu \Delta s - \frac{1}{2} \Delta\kappa_\nu (\Delta s)^2} \right) - \frac{\epsilon_\nu \Delta\kappa_\nu - \kappa_\nu \Delta\epsilon_\nu}{\Delta\kappa_\nu} \sqrt{\frac{\pi}{2|\Delta\kappa_\nu|}} \left( e^{a_\nu^2 - b_\nu^2}\tilde{E}(a_\nu) - \tilde{E}(b_\nu) \right) + I_{\nu,0} e^{-\kappa_\nu \Delta s - \frac{1}{2} \Delta\kappa_\nu (\Delta s)^2}
$$

Here $a_\nu=\frac{\kappa_\nu}{\sqrt{2\Delta\kappa_\nu}}$ and $b_\nu=\frac{\kappa_\nu + \Delta\kappa_\nu \Delta s}{\sqrt{2 \Delta\kappa_\nu}}$, while $\tilde{E}$ is an approximation of the imaginary error function. The distinction we make is how we treat the error function if $a,b$ are real or imaginary values.

### Calculation

With the radiative transfer method focused on the offending sightlines, we can obtain these values used in the RT equation. Now it is simply a matter of re-evaluating manually to compare the results (and find any relevant error). Since our model of interest has voxels with size $s=400pc$, we have $\Delta s = s [cm]$.

In [4]:
import numpy as np

ds = 1.2276119715538785e20# 400 * 3.086*10**18

test_intensity = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\I.npy')
test_epsilon = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\e.npy')
test_depsilon = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\de.npy')
test_kappa = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\k.npy')
test_dkappa = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\dk.npy')
test_a = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\a.npy')
test_b = np.load(r'c:\users\cyani\KOSMA-tau^3\tests\full model\b.npy')

In [5]:
from scipy import interpolate

GRIDPATH = r'c:\users\cyani\KOSMA-tau^3\kosmatau3d\grid'

def eTildeRealData(file='\Ereal.dat'):
    ereal = np.genfromtxt(GRIDPATH+file, names=['x', 'Ereal'])
    return (ereal['x'], ereal['Ereal'])


def eTildeImaginaryData(file='\Eimag.dat'):
    eimaginary = np.genfromtxt(GRIDPATH+file, names=['x', 'Eimaginary'])
    return (eimaginary['x'], eimaginary['Eimaginary'])

eTildeRealObs = eTildeRealData()
eTildeImaginaryObs = eTildeImaginaryData()

eTildeReal = interpolate.interp1d(eTildeRealObs[0], eTildeRealObs[1], kind='linear')
eTildeImaginary = interpolate.interp1d(eTildeImaginaryObs[0], eTildeImaginaryObs[1], kind='linear')

def Ereal(x, test=False):
  
    Er = 0
    
    if test is True:
        Er = np.zeros(x.size)
        il = x < 0.01
        ig = (~il) & (x > 8.0)
        ib = ~(ig | il)
        if (il | ig | ib).all():
            if il.any():
                Er[il] = (2*x[il]/np.sqrt(np.pi))
            if ig.any():
                Er[ig] = (1/(np.sqrt(np.pi) * x[ig]))
            if ib.any():
                Er[ib] = eTildeReal(x[ib])
        else:
            print('Ereal() error: incorrect input')
    
    elif test is False:
        if x < 0.01:
            Er = (2*x/np.sqrt(np.pi))
        elif x > 8.0:
            Er = (1/(np.sqrt(np.pi) * x))
        else:
            Er = eTildeReal(x)
          
    return Er

def Eimag(x, test=False):
  
    Ei = 0
    
    if test is True:
        Ei = np.zeros(x.size)
        im = x > 0
        il = ~im & (abs(x) < 0.01)
        ig = ~im & (abs(x) > 8.0)
        ib = ~(ig | il | im)
        if (im|il|ig|ib).all():
            if im.any():
                # MASER case: treat with linear approximation
                Ei[im] = 1 + 2*np.abs(x[im])/np.sqrt(np.pi)
            if il.any():
                Ei[il] = (1 - 2*np.abs(x[il])/np.sqrt(np.pi))
            if ig.any():
                Ei[ig] = (1/(np.sqrt(np.pi) * np.abs(x[ig])))
            if ib.any():
                Ei[ib] = eTildeImaginary(np.abs(x[ib]))
        else:
            print('Eimag() error: incorrect input')
            
    elif test is False:
        if x > 0:
            # MASER case: treat with linear approximation
            Ei = 1 + 2*np.abs(x)/np.sqrt(np.pi)
        elif (x < 0) and (np.abs(x) < 0.01):
            Ei = (1 - 2*np.abs(x)/np.sqrt(np.pi))
        elif (x == (-1j*np.abs(x))) and (np.abs(x) > 8.0):
            Ei = (1/(np.sqrt(np.pi) * np.abs(x)))
        else:
            Ei = eTildeImaginary(np.abs(x))
          
    return Ei

def rt_basic(i, vel, spe, intensity, PRINT=0):
    if PRINT > 0:
        print(1)
    return test_epsilon[i, spe]*ds \
           + 0.5*test_depsilon[i, spe]*ds**2. \
           + intensity

def rt_simple(i, vel, spe, intensity, PRINT=0):
    if PRINT > 0:
        print(2)
    return ((test_epsilon[i, spe]*test_kappa[i, spe] + test_depsilon[i, spe]*(test_kappa[i, spe]*ds-1.))
            /test_kappa[i, spe]**2.) \
           - np.exp(-test_kappa[i, spe]*ds) \
           * ((test_epsilon[i, spe]*test_kappa[i, spe] 
               - test_depsilon[i, spe])
              /test_kappa[i, spe]**2.) \
           + np.exp(-test_kappa[i, spe]*ds)*intensity

def rt_complex1(i, vel, spe, intensity, test=False, PRINT=0):
    if PRINT > 0:
        print(3)
    return (test_depsilon[i, spe]/test_dkappa[i, spe]
            * (1-np.exp(-test_kappa[i, spe]*ds - 0.5*test_dkappa[i, spe]*ds**2.))
            - (test_epsilon[i, spe]*test_dkappa[i, spe] - test_kappa[i, spe]*test_depsilon[i, spe])/test_dkappa[i, spe]
            * np.sqrt(np.pi/2./np.abs(test_dkappa[i, spe]))
            * (np.exp(test_a[i, spe]**2.-test_b[i, spe]**2.).real*Ereal(test_a[i, spe].real, test=test) 
                      - Ereal(test_b[i, spe].real, test=test)) 
            + intensity*np.exp(-test_kappa[i, spe]*ds - 0.5*test_dkappa[i, spe]*ds**2.))

def rt_complex2(i, vel, spe, intensity, test=False, PRINT=0):
    if PRINT > 0:
        print(4)
    return (test_depsilon[i, spe]/test_dkappa[i, spe]
            * (1-np.exp(-test_kappa[i, spe]*ds - 0.5*test_dkappa[i, spe]*ds**2.)) 
            - (test_epsilon[i, spe]*test_dkappa[i, spe] - test_kappa[i, spe]*test_depsilon[i, spe])/test_dkappa[i, spe]
            * np.sqrt(np.pi/2./np.abs(test_dkappa[i, spe]))
            * (np.exp(test_a[i, spe]**2.-test_b[i, spe]**2).real*Eimag(test_a[i, spe].imag, test=test) 
                      - Eimag(test_b[i, spe].imag, test=test)) 
            + intensity*np.exp(-test_kappa[i, spe]*ds - 0.5*test_dkappa[i, spe]*ds**2.))


In [6]:
spe = 'all'
vel = 0
test = False
PRINT = 0

if type(spe) == int:
    
    if PRINT > 1:
        print('single species')
    
    intensity = 0

elif spe == 'all':
    
    if PRINT > 1:
        print('all species')
    
    intensity = np.zeros(test_depsilon.shape[1])

for i in range(test_depsilon.shape[0]):
  
    if type(spe) == int:

        k0 = np.asarray((test_dkappa[i, spe] == 0) and (np.abs(test_kappa[i, spe]*ds) < 10**-10))
        kg = np.asarray((test_dkappa[i, spe] == 0) or (np.abs(test_kappa[i, spe]) > np.abs(10**3*test_dkappa[i, spe]*ds)))
        keg = np.asarray(test_dkappa[i, spe] > 0)
        kel = np.asarray(test_dkappa[i, spe] < 0)
    
    elif spe == 'all':

        k0 = np.asarray((test_dkappa[i, :] == 0) & (np.abs(test_kappa[i, :]*ds) < 10**-10))
    #     kg = np.asarray(~k0 & ((test_dkappa[i, vel, :] == 0) | (np.abs(test_kappa[i, vel, :]) > np.abs(10**3*test_dkappa[i, vel, :]*ds))))
        kg = np.asarray(~k0 & ((np.abs(test_kappa[i, :]) > np.abs(10**3*test_dkappa[i, :]*ds))))
        keg = np.asarray(~k0 & ~kg & (test_dkappa[i, :] > 0))
        kel = np.asarray(~k0 & ~kg & (test_dkappa[i, :] < 0))
    
    else:
    
        print('species not recognised')
        exit()
    
#   np.set_printoptions(precision=16)
#     print(intensity[0])
  
    if k0.any():
    
        if PRINT > 0:
#       print(1)
            if PRINT == 3:
                print('Intensity:', intensity)
                print('epsilon:', test_epsilon[i], test_depsilon[i])
                print('kappa:', test_kappa[i], test_dkappa[i])
        
        if type(spe) == int:
            intensity = rt_basic(i, vel, spe, intensity, PRINT=PRINT)
      
        elif spe == 'all':
            intensity[k0] = rt_basic(i, vel, k0, intensity[k0], PRINT=PRINT)
      
    elif kg.any():

        if PRINT > 0:
    #       print(2)
            if PRINT == 3:
                print('Intensity:', intensity)
                print('epsilon:', test_epsilon[i], test_depsilon[i])
                print('kappa:', test_kappa[i], test_dkappa[i])
                print(10**3*test_dkappa[i]*ds)

        if type(spe) == int:
            intensity = rt_simple(i, vel, spe, intensity, PRINT=PRINT)

        elif spe == 'all':
            intensity[kg] = rt_simple(i, vel, kg, intensity[kg], PRINT=PRINT)
      
    else:
      # It is a bit redundent to have this else statement containing another if statement, 
      #  but this is done for clarity of the integration case.
      if keg.any():

          if PRINT > 0:
    #         print(3)
              if PRINT == 3:
                  print('Intensity:', intensity)
                  print('epsilon:', test_epsilon[i], test_depsilon[i])
                  print('kappa:', test_kappa[i], test_dkappa[i])

          if type(spe) == int:
              intensity = rt_complex1(i, vel, spe, intensity, test=False, PRINT=PRINT)

          elif spe == 'all':
              intensity[keg] = rt_complex1(i, vel, keg, intensity[keg], test=True, PRINT=PRINT)

      elif kel.any():

          if PRINT > 0:
      #         print(4)
              if PRINT == 3:
                  print('Intensity:', intensity)
                  print('epsilon:', test_epsilon[i], test_depsilon[i])
                  print('kappa:', test_kappa[i], test_dkappa[i])

          if type(spe) == int:
              intensity = rt_complex2(i, vel, spe, intensity, test=False, PRINT=PRINT)

          elif spe == 'all':
              intensity[kel] = rt_complex2(i, vel, kel, intensity[kel], test=True, PRINT=PRINT)

      else:

          print('Error!')

if type(spe) == int:
    print('\n Old intensity:', test_intensity[spe])
elif spe == 'all':
    print('\n Old intensity:', test_intensity)
print('Test intensity:', intensity)


 Old intensity: [-1.41568931]
Test intensity: [-1.41568931]


In [19]:
a = ~ np.asarray([True, False])

array([ True, False])

In [73]:
test_epsilon[300,0] == test_epsilon[300,:]

array([ True,  True,  True,  True])

In [30]:
test_intensity

array([-1.00452326e+00, -4.63788106e-02, -2.19011849e-02, -1.35091614e-01,
       -1.61151223e-01, -8.17112877e-02, -3.11454402e-02, -8.40420609e-04,
       -2.58992159e-04, -3.37617615e-02, -2.17602348e-03])

In [63]:
a = np.random.rand(10)
b = np.random.rand(10)
print(a > 0.5)
print(b < 0.2)
print(~(a>0.5) & ~(b<0.2))

[False False  True  True False  True  True  True  True False]
[False False  True False False False  True False False False]
[ True  True False False  True False False False False  True]
